<a href="https://colab.research.google.com/github/juyeon-shin/CLUB-D.N.A./blob/main/2022/05.%20%EB%8B%A4%EC%9D%B4%EC%95%84%EB%AA%AC%EB%93%9C%20%EA%B0%80%EA%B2%A9%20%EC%98%88%EC%B8%A1/220823_%EC%A3%BC%EC%97%B0_%EB%B3%80%EC%88%98%EC%84%A0%ED%83%9D.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd
import numpy as np

import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
train = pd.read_csv('/content/drive/Shareddrives/202206 나의 분석일지/다이아몬드 가격 예측/Diamonds Prices2022.csv')

In [ ]:
train = train.drop(['Unnamed: 0'],axis=1)
train

,carat,cut,color,clarity,depth,table,price,x,y,z
0,0.23,Ideal,E,SI2,61.5,55.0,326,3.95,3.98,2.43
1,0.21,Premium,E,SI1,59.8,61.0,326,3.89,3.84,2.31
2,0.23,Good,E,VS1,56.9,65.0,327,4.05,4.07,2.31
3,0.29,Premium,I,VS2,62.4,58.0,334,4.20,4.23,2.63
4,0.31,Good,J,SI2,63.3,58.0,335,4.34,4.35,2.75
...,...,...,...,...,...,...,...,...,...,...
53938,0.86,Premium,H,SI2,61.0,58.0,2757,6.15,6.12,3.74
53939,0.75,Ideal,D,SI2,62.2,55.0,2757,5.83,5.87,3.64
53940,0.71,Premium,E,SI1,60.5,55.0,2756,5.79,5.74,3.49
53941,0.71,Premium,F,SI1,59.8,62.0,2756,5.74,5.73,3.43


### ANOVA 분석을 위한 가정 검정
+ 정규성 : shapiro
+ 독립성
+ 등분산성 : levene, bartlett

In [ ]:
import scipy.stats

#### 정규성 검정
+ 각 범주형 변수의 요소들의 데이터 수가 30이상이라 정규성 검정이 필요하지는 않음
+ 결과가 궁금해서 한 번 실행

In [ ]:
cat_list = ['cut','color','clarity']

In [ ]:
for cat in cat_list:
  print(train.groupby(['%s'%cat]).count())

           carat  color  clarity  depth  table  price      x      y      z
cut                                                                       
Fair        1610   1610     1610   1610   1610   1610   1610   1610   1610
Good        4906   4906     4906   4906   4906   4906   4906   4906   4906
Ideal      21551  21551    21551  21551  21551  21551  21551  21551  21551
Premium    13793  13793    13793  13793  13793  13793  13793  13793  13793
Very Good  12083  12083    12083  12083  12083  12083  12083  12083  12083
       carat    cut  clarity  depth  table  price      x      y      z
color                                                                 
D       6775   6775     6775   6775   6775   6775   6775   6775   6775
E       9799   9799     9799   9799   9799   9799   9799   9799   9799
F       9543   9543     9543   9543   9543   9543   9543   9543   9543
G      11292  11292    11292  11292  11292  11292  11292  11292  11292
H       8304   8304     8304   8304   8304   8304

In [ ]:
for cat in cat_list:
  print('shapiro : %s'%cat)
  for uni in train[cat].unique():
    print('%s: '%uni,scipy.stats.shapiro(train.price[train[cat]== uni]))

shapiro : cut
Ideal:  ShapiroResult(statistic=0.7493794560432434, pvalue=0.0)
Premium:  ShapiroResult(statistic=0.8345707058906555, pvalue=0.0)
Good:  ShapiroResult(statistic=0.8170838952064514, pvalue=0.0)
Very Good:  ShapiroResult(statistic=0.8136837482452393, pvalue=0.0)
Fair:  ShapiroResult(statistic=0.8102028965950012, pvalue=6.6015310784188565e-40)
shapiro : color
E:  ShapiroResult(statistic=0.7287397384643555, pvalue=0.0)
I:  ShapiroResult(statistic=0.8536186814308167, pvalue=0.0)
J:  ShapiroResult(statistic=0.8894203305244446, pvalue=6.114986238620637e-41)
H:  ShapiroResult(statistic=0.8413457870483398, pvalue=0.0)
F:  ShapiroResult(statistic=0.7838588953018188, pvalue=0.0)
G:  ShapiroResult(statistic=0.8052399754524231, pvalue=0.0)
D:  ShapiroResult(statistic=0.7435647249221802, pvalue=0.0)
shapiro : clarity
SI2:  ShapiroResult(statistic=0.8306176662445068, pvalue=0.0)
SI1:  ShapiroResult(statistic=0.8130813837051392, pvalue=0.0)
VS1:  ShapiroResult(statistic=0.792882800102233

/usr/local/lib/python3.7/dist-packages/scipy/stats/morestats.py:1760: UserWarning: p-value may not be accurate for N > 5000.
  warnings.warn("p-value may not be accurate for N > 5000.")


+ 정규성 검정 결과 정규성 자체를 만족시키진 않음 (p-value >=0.05)
+ 데이터 수 자체가 많기 때문에 정규성을 만족시킨다 결론

#### 독립성 검정
+ 자료 수집이 랜덤 샘플링이라면 만족한다고 봄

#### 등분산성 검정
+ 모든 집단의 모분산은 동일하다 가정
+ 집단의 모분산의 유의미한 차이 파악
+ 귀무가설 : 해당 범주형 변수의 등분산은 모두 같다

In [ ]:
cat = 'cut'

scipy.stats.levene(
      train.price[train[cat] == train[cat].unique()[0]],
      train.price[train[cat] == train[cat].unique()[1]],
      train.price[train[cat] == train[cat].unique()[2]],
      train.price[train[cat] == train[cat].unique()[3]],
      train.price[train[cat] == train[cat].unique()[4]],
      #train.price[train[cat] == train[cat].unique()[5]],
      #train.price[train[cat] == train[cat].unique()[6]],
      #train.price[train[cat] == train[cat].unique()[7]]
  )

LeveneResult(statistic=123.45971747697452, pvalue=4.390887391642353e-105)

In [ ]:
cat = 'color'

scipy.stats.levene(
      train.price[train[cat] == train[cat].unique()[0]],
      train.price[train[cat] == train[cat].unique()[1]],
      train.price[train[cat] == train[cat].unique()[2]],
      train.price[train[cat] == train[cat].unique()[3]],
      train.price[train[cat] == train[cat].unique()[4]],
      train.price[train[cat] == train[cat].unique()[5]],
      #train.price[train[cat] == train[cat].unique()[6]],
      #train.price[train[cat] == train[cat].unique()[7]]
  )

LeveneResult(statistic=206.70928118075773, pvalue=8.425148598413593e-219)

In [ ]:
cat = 'clarity'

scipy.stats.levene(
      train.price[train[cat] == train[cat].unique()[0]],
      train.price[train[cat] == train[cat].unique()[1]],
      train.price[train[cat] == train[cat].unique()[2]],
      train.price[train[cat] == train[cat].unique()[3]],
      train.price[train[cat] == train[cat].unique()[4]],
      train.price[train[cat] == train[cat].unique()[5]],
      train.price[train[cat] == train[cat].unique()[6]],
      train.price[train[cat] == train[cat].unique()[7]]
  )

LeveneResult(statistic=77.7954880190435, pvalue=7.954171566013717e-113)

+ 범주형 변수의 등분산성 검정 결과 모두 p-value 0.05보다 작아서 대립가설을 지지

-> 각 범주형 변수는 등분산성을 만족하지 않는다

### ANOVA 분석
+ 등분산성을 만족하지는 않지만 일단 진행
+ 귀무가설 : 범주형 변수들의 각 항목의 분산은 같다

In [ ]:
from statsmodels.formula.api import ols
from statsmodels.stats.anova import anova_lm

In [ ]:
model = ols('price ~ C(cut)',train).fit()
anova_lm(model,typ=3)

,sum_sq,df,F,PR(>F)
Intercept,3.058802e+10,1.0,1946.872192,0.000000e+00
C(cut),1.103771e+10,4.0,175.632635,9.410997e-150
Residual,8.474396e+11,53938.0,NaN,NaN


In [ ]:
model = ols('price ~ C(color)',train).fit()
anova_lm(model,typ=3)

,sum_sq,df,F,PR(>F)
Intercept,6.807933e+10,1.0,4415.362419,0.0
C(color),2.685212e+10,6.0,290.254142,0.0
Residual,8.316252e+11,53936.0,NaN,NaN


In [ ]:
model = ols('price ~ C(clarity)',train).fit()
anova_lm(model,typ=3)

,sum_sq,df,F,PR(>F)
Intercept,1.141073e+10,1.0,736.901535,3.449321e-161
C(clarity),2.330752e+10,7.0,215.027432,1.865558e-316
Residual,8.351698e+11,53935.0,NaN,NaN


In [ ]:
model = ols('price ~ C(cut) * C(color) * C(clarity)',train).fit()
anova_lm(model,typ=3)

,sum_sq,df,F,PR(>F)
Intercept,2.180348e+08,1.0,14.988717,1.082840e-04
C(cut),5.985246e+07,4.0,1.028634,3.907378e-01
C(color),2.541475e+08,6.0,2.911879,7.705442e-03
C(clarity),7.482101e+07,7.0,0.734792,6.424520e-01
C(cut):C(color),3.027121e+08,24.0,0.867076,6.499033e-01
C(cut):C(clarity),1.025042e+09,28.0,2.516647,1.616571e-05
C(color):C(clarity),1.445712e+09,42.0,2.366308,1.515651e-06
C(cut):C(color):C(clarity),5.333367e+09,168.0,2.182383,9.449956e-17
Residual,7.806720e+11,53667.0,NaN,NaN


#### 통계적으로 유의한 변수 (p-value < 0.05)
+ color
+ C(cut):C(clarity)
+ C(color):C(clarity)
+ C(cut):C(color):C(clarity)

### 유의한 변수에 대한 사후분석
+ ANOVA 검증 결과, 유의미하다는 결론을 얻었을때, 구체적으로 어떤 수준에서 평균 차이가 나는지 검증
+ 투키의 HSD 시행
+ 통계적으로 유의한 변수를 이용해 파생변수 생성 후 사후분석 시행

In [ ]:
train_c = train.copy()
train_c

,carat,cut,color,clarity,depth,table,price,x,y,z
0,0.23,Ideal,E,SI2,61.5,55.0,326,3.95,3.98,2.43
1,0.21,Premium,E,SI1,59.8,61.0,326,3.89,3.84,2.31
2,0.23,Good,E,VS1,56.9,65.0,327,4.05,4.07,2.31
3,0.29,Premium,I,VS2,62.4,58.0,334,4.20,4.23,2.63
4,0.31,Good,J,SI2,63.3,58.0,335,4.34,4.35,2.75
...,...,...,...,...,...,...,...,...,...,...
53938,0.86,Premium,H,SI2,61.0,58.0,2757,6.15,6.12,3.74
53939,0.75,Ideal,D,SI2,62.2,55.0,2757,5.83,5.87,3.64
53940,0.71,Premium,E,SI1,60.5,55.0,2756,5.79,5.74,3.49
53941,0.71,Premium,F,SI1,59.8,62.0,2756,5.74,5.73,3.43


In [ ]:
train_c['cut/clarity'] = train_c['cut']+'/'+train_c['clarity']
train_c['color/clarity'] = train_c['color']+'/'+train_c['clarity']
train_c['cut/color/clarity'] = train_c['cut']+'/'+train_c['color']+'/'+train_c['clarity']
train_c

,carat,cut,color,clarity,depth,table,price,x,y,z,cut/clarity,color/clarity,cut/color/clarity
0,0.23,Ideal,E,SI2,61.5,55.0,326,3.95,3.98,2.43,Ideal/SI2,E/SI2,Ideal/E/SI2
1,0.21,Premium,E,SI1,59.8,61.0,326,3.89,3.84,2.31,Premium/SI1,E/SI1,Premium/E/SI1
2,0.23,Good,E,VS1,56.9,65.0,327,4.05,4.07,2.31,Good/VS1,E/VS1,Good/E/VS1
3,0.29,Premium,I,VS2,62.4,58.0,334,4.20,4.23,2.63,Premium/VS2,I/VS2,Premium/I/VS2
4,0.31,Good,J,SI2,63.3,58.0,335,4.34,4.35,2.75,Good/SI2,J/SI2,Good/J/SI2
...,...,...,...,...,...,...,...,...,...,...,...,...,...
53938,0.86,Premium,H,SI2,61.0,58.0,2757,6.15,6.12,3.74,Premium/SI2,H/SI2,Premium/H/SI2
53939,0.75,Ideal,D,SI2,62.2,55.0,2757,5.83,5.87,3.64,Ideal/SI2,D/SI2,Ideal/D/SI2
53940,0.71,Premium,E,SI1,60.5,55.0,2756,5.79,5.74,3.49,Premium/SI1,E/SI1,Premium/E/SI1
53941,0.71,Premium,F,SI1,59.8,62.0,2756,5.74,5.73,3.43,Premium/SI1,F/SI1,Premium/F/SI1


In [ ]:
from statsmodels.stats.multicomp import pairwise_tukeyhsd

In [ ]:
def make_hsd (col1,col2,alpha):
  hsd = pairwise_tukeyhsd(col1, col2,alpha=alpha)
  hsd_df = pd.DataFrame()
  #hsd_df['group1'] = hsd.groups
  hsd_df['reject'] = hsd.reject
  hsd_df['pvalues'] = hsd.pvalues
  return hsd_df.groupby(['reject']).count(), hsd.summary()

In [ ]:
make_hsd(train_c['price'],train_c['color'],0.05)[0]

,pvalues
reject,
False,2
True,19


In [ ]:
make_hsd(train_c['price'],train_c['color'],0.05)[1]

group1,group2,meandiff,p-adj,lower,upper,reject
D,E,-93.267,0.7156,-276.1943,89.6603,False
D,F,554.8308,0.001,370.9032,738.7583,True
D,G,829.1816,0.001,651.2662,1007.0969,True
D,H,1316.7151,0.001,1127.1762,1506.254,True
D,I,1921.9209,0.001,1710.9597,2132.882,True
D,J,2153.8639,0.001,1894.0227,2413.7051,True
E,F,648.0978,0.001,481.5926,814.603,True
E,G,922.4486,0.001,762.6096,1082.2876,True
E,H,1409.9821,0.001,1237.2984,1582.6657,True
E,I,2015.1878,0.001,1819.2306,2211.1451,True


In [ ]:
make_hsd(train_c['price'],train_c['cut/clarity'],0.05)[0]

,pvalues
reject,
False,502
True,278


In [ ]:
make_hsd(train_c['price'],train_c['color/clarity'],0.05)[0]

,pvalues
reject,
False,714
True,826


In [ ]:
make_hsd(train_c['price'],train_c['cut/color/clarity'],0.05)[0]

,pvalues
reject,
False,32634
True,5316


#### 사후분석 결과
+ p-values가 0.05보다 작으면 True -> 유의하다
+ color : 대부분의 집단에서 평균의 차이가 존재함
+ cut/clarity : 특정 집단에서 평균 차이 존재
+ color/clarity : 특정 집단에서 평균 차이 존재
+ cut/color/clarity : 특정 집단에서 평균 차이 존재

### 최종 변수 선택
+ 수치형 변수 : carat, x,y,z
+ 범주형 변수 : color, color/clarity

color/clarity 는 파생변수